## 預載資料

In [ ]:
import requests
import pandas
import sqlite3 as lite
from json import JSONDecoder
#import cv2
import time,os
import numpy as np
import random
import sys,os
import configparser as ConfigParser
import requests
import pandas
import sqlite3 as lite
from json import JSONDecoder
import os,shutil,time
import numpy as np  

ini_data=ConfigParser.ConfigParser()
ini_data.read('key.ini')

key = ini_data.get('facepp','key')
secret = ini_data.get('facepp','secret')


## 創建faceset

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/create"
data = {"api_key": key, "api_secret": secret, 'display_name':'FacesStore',"outer_id":"line_bot_av_girl"}

response = requests.post(http_url, data=data)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(response)
print(req_dict)

## 製做facetoken

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/detect"

name=[input('請輸入女優的名子：')]#人物名子
ps=[input('請輸入女優的附註：')]
filepath = input('請輸入女優的檔名：')+'.jpg'#人物檔案位置_一定要羅馬拼音
logo=[input('請輸入logo網址：')]


#######################先上傳判斷############################
decide=1#try跳開
count=0#失敗計數數量
while decide:
    time.sleep(1)
    try:
        data = {"api_key": key, "api_secret": secret}
        files = {"image_file": open(filepath, "rb")}
        response = requests.post(http_url, data=data, files=files)

        req_con = response.content.decode('utf-8')
        req_dict = JSONDecoder().decode(req_con)

        print(req_dict)
       
        decide=0
    except KeyError as e3:
        count=count+1
        print('field')
        if count==8:
            decide=0

## 寫入資料庫程序

In [ ]:
pinyin=[filepath[0:filepath.find('.jpg')]]
face_token=[req_dict['faces'][0]['face_token']]

av_data={'名子':name,'羅馬拼音':pinyin,'face_tokens':face_token,'附註':ps,'logo':logo}
write=pandas.DataFrame(av_data)

with lite.connect('av_data.sqlite') as db:
    write.to_sql('face_search',con=db,if_exists='append')
    
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)

## 在faceset裡面新增臉

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/addface"

data = {"api_key": key, "api_secret": secret, "outer_id": "line_bot_av_girl","face_tokens":'b50e099035902daff0a90213bf2be061'}

response = requests.post(http_url, data=data)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)

print(response)
print(response.text)

## 在faceset搜索

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/search"

filepath = "test.jpg"

data = {"api_key": key, "api_secret": secret, "outer_id": "line_bot_av_girl","return_result_count":5}
files = {"image_file": open(filepath, "rb")}
response = requests.post(http_url, data=data, files=files)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(req_dict)


#與資料庫名子匹配
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)
ans_name=read[read["face_tokens"]==req_dict['results'][0]['face_token']].iloc[0,2]    
if req_dict['results'][0]['confidence']>83:
    word="我覺得是"+ans_name+"啦\n"+"相似程度有%f"%req_dict['results'][0]['confidence']
else:
    word="找不到相似的人 但可能是"+ans_name+"\n"+"相似程度有%f"%req_dict['results'][0]['confidence']

## 搜尋全部的Faceset

In [ ]:
http_url='https://api-cn.faceplusplus.com/facepp/v3/faceset/getfacesets'

data={"api_key": key, "api_secret": secret}

response = requests.post(http_url, data=data)
req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(response)

In [ ]:
req_dict

## 取得faceset列表

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/getdetail"

data = {"api_key": key, "api_secret": secret, "outer_id": input('請輸入faceset名稱：'),'start':int(input('請輸入起始數字：'))}

response = requests.post(http_url, data=data)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(response)
print('檔案總數：'+str(req_dict['face_count']))
print('回傳tokens數量：'+str(len(req_dict['face_tokens'])))



## 伺服器比對電腦器

In [ ]:
count=0
check_face_tokens=[]
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)
for item_s in req_dict['face_tokens']:
    for item_c in read['face_tokens'].tolist():
        if item_s==item_c:
            #print('電腦的:'+item_c+'||伺服器的:'+item_s)
            count=count+1
            check_face_tokens+=[item_s]
print('電腦數量：'+str(len(read['face_tokens'].tolist()))+'伺服器數量：'+str(req_dict['face_count']))
for item in check_face_tokens:
    req_dict['face_tokens'].remove(item)
print('計數次數：'+str(count))
for item in req_dict['face_tokens']:
    print('錯誤檔案：'+item)

## 刪除faceset

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/delete"

data = {"api_key": key, "api_secret": secret, "outer_id": input('請輸入要刪除的faceset名稱：'),"check_empty":0}

response = requests.post(http_url, data=data)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(req_dict)




## 刪除face

In [ ]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/removeface"

data = {"api_key": key, "api_secret": secret, "face_tokens": input('請輸入facetoken：'),'outer_id':'line_bot_av_girl'}

response = requests.post(http_url, data=data)

req_con = response.content.decode('utf-8')
req_dict = JSONDecoder().decode(req_con)
print(req_dict)




## 以雲端表單新增女優資料
使用email_python-to-excel@python-to-excel.iam.gserviceaccount.com

In [ ]:
import math

from imgurpython import ImgurClient#上傳圖片程序
import time

#畫圖程序
import matplotlib.pyplot as plt
import numpy as np
#畫圖程序

import example#c++連結
import ctypes
import pandas
import requests
from bs4 import BeautifulSoup

import googlemaps
from datetime import datetime

from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError,LineBotApiError
)
from linebot.models import (
    ImageCarouselColumn,ImageCarouselTemplate,CarouselColumn,CarouselTemplate,ImageMessage,MessageEvent, TextMessage, TextSendMessage,LocationMessage,FollowEvent,PostbackEvent,ImageSendMessage,FollowEvent,TemplateSendMessage,ButtonsTemplate,PostbackTemplateAction,MessageTemplateAction,URITemplateAction
)

#圖片辨識流程by face++
import requests
import pandas
import sqlite3 as lite
from json import JSONDecoder
#圖片辨識流程by face++

import os,shutil,cv2#資料移動相關

####excel插件
import sys
import time
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials as SAC
####excel插件

In [ ]:
GDriveJSON = 'python to excel-c35aea97a97b.json'
GSpreadSheet = 'AV女優資料庫'

try:
    scope = ['https://spreadsheets.google.com/feeds']
    key = SAC.from_json_keyfile_name(GDriveJSON, scope)
    gc = gspread.authorize(key)
    sh = gc.open('AV女優資料庫')

except Exception as ex:
    print('無法連線Google試算表', ex)
    sys.exit(1)

In [ ]:
worksheet = sh.worksheet("2010年出道 AV女優列表")

In [ ]:
for_excel_count=worksheet.range('B52:B52')

In [ ]:
len(for_excel_count)

In [ ]:
excel_count=0
for item in for_excel_count:
    print(excel_data[0+excel_count].value+'_'+excel_data[1+excel_count].value+'_'+excel_data[2+excel_count].value)
    
    cli_name=excel_data[0+excel_count].value
    cli_av_file_name=excel_data[1+excel_count].value
    cli_web=excel_data[2+excel_count].value
    
    excel_count=excel_count+3

In [ ]:
excel_count=0
for item in for_excel_count:
    print(excel_data[0+excel_count].value+'_'+excel_data[1+excel_count].value+'_'+excel_data[2+excel_count].value)
    
    cli_name=excel_data[0+excel_count].value
    cli_av_file_name=excel_data[1+excel_count].value
    cli_web=excel_data[2+excel_count].value
    
    excel_count=excel_count+3
    #建立女優資料##################################
    key = "B2LK45DwbvhBhz6NFcja5q4m3182NMmk"
    secret = "h3jMXHC-fc0aXdcuo_cQOQBERuICfZHA"

    name=[cli_name]#人物名子
    ps=['']#人物附註
    av_file_name=cli_av_file_name+'.jpg'#人物檔名 一定要羅馬拼音

    #製作女優資料夾
    try:
        os.mkdir('av_girl_data/'+name[0])
    except FileExistsError as e:
        None
    #製作女優資料夾

    #製作女優圖片
    response=requests.get(cli_web,stream=True)
    print(response)

    with open(av_file_name,'wb') as f:
        f.write(response.content)
    #製作女優圖片

    #製作女優圖片檔名排序
    dir_file=os.listdir('av_girl_data/'+name[0])
    file_name=[]
    for item in dir_file:
        file_name+=[int(item[item.find('_')+1:item.find('.jpg')])]
    if file_name==[]:
        file_name=[0]
    #製作女優圖片檔名排序

    #移動圖片
    file_end='av_girl_data/'+name[0]+'/'+av_file_name[0:av_file_name.find('.jpg')]+'_'+str(max(file_name)+1)+'.jpg'
    shutil.move(av_file_name,file_end)
    #移動圖片

    ##########################################################################

    #製作facetoken######################先上傳判斷############################
    http_url = "https://api-cn.faceplusplus.com/facepp/v3/detect"

    decide=1#try跳開
    count=0#失敗計數數量
    while decide:
        time.sleep(1)
        try:
            with  open(file_end, "rb") as f:
                data = {"api_key": key, "api_secret": secret}
                files = {"image_file": f}
                response = requests.post(http_url, data=data, files=files)

                req_con = response.content.decode('utf-8')
                req_dict = JSONDecoder().decode(req_con)

                print(req_dict)
                print('人臉數量'+str(len(req_dict['faces'])))
                decide=0
        except KeyError as e3:
            count=count+1
            print('field')
            if count==8:
                decide=0
    #######################先上傳判斷############################


    #######################將剪裁上傳############################
    if len(req_dict['faces'])==1:#判斷圖片是否只有一人
        left=req_dict['faces'][0]['face_rectangle']['left']
        top=req_dict['faces'][0]['face_rectangle']['top']
        width=req_dict['faces'][0]['face_rectangle']['width']
        height=req_dict['faces'][0]['face_rectangle']['height']

        tri=10#方框加回來

        #讓opencv可以讀中文路徑
        def cv_imread(filePath):  
            cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)  
            ## imdecode读取的是rgb，如果后续需要opencv处理的话，需要转换成bgr，转换后图片颜色会变化  
            ##cv_img=cv2.cvtColor(cv_img,cv2.COLOR_RGB2BGR)  
            return cv_img  
        #讓opencv可以讀中文路徑

        #讀取
        image = cv_imread(file_end)
        #剪裁 因為錯誤剪取所以取消
        #cv2.imencode('.jpg',image[top-tri: top+height+tri,left-tri:left+width+tri])[1].tofile(file_end)  
        #剪裁後讀取 因為錯誤剪取所以取消
        #image = cv_imread(file_end)

        decide=1#try跳開
        count=0#失敗計數數量
        while decide:
            time.sleep(1)
            try:
                data = {"api_key": key, "api_secret": secret}
                files = {"image_file": open(file_end, "rb")}
                response = requests.post(http_url, data=data, files=files)

                req_con = response.content.decode('utf-8')
                req_dict = JSONDecoder().decode(req_con)

                print(req_dict)
                print('人臉數量'+str(len(req_dict['faces'])))
                decide=0
            except KeyError as e3:
                count=count+1
                print('field')
                if count==8:
                    decide=0
        ##製作facetoken######################將剪裁上傳############################

        #紀錄資料庫程序
        pinyin=[av_file_name[0:av_file_name.find('.jpg')]]
        face_token=[req_dict['faces'][0]['face_token']]
        
        #logo檢查程序 如果有就增加 沒有就空白
        with lite.connect('av_data.sqlite') as db:
            read=pandas.read_sql_query('select * from face_search',con = db)
            if sum(read["名子"]==name[0])!=0:
                logo=read[read["名子"]==name[0]].iloc[0,6]
            else:
                logo=''
        #logo檢查程序 如果有就增加 沒有就空白

        av_data={'名子':name,'羅馬拼音':pinyin,'face_tokens':face_token,'附註':ps,'檔名':[av_file_name[0:av_file_name.find('.jpg')]+'_'+str(max(file_name)+1)+'.jpg'],'logo':[logo]}
        write=pandas.DataFrame(av_data)

        print('=======================================')

        #在faceset裡面新增臉
        http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/addface"

        decide=1#try跳開
        count=0#失敗計數數量
        while decide:
            time.sleep(1)
            try:
                data = {"api_key": key, "api_secret": secret, "outer_id": "line_bot_av_girl","face_tokens":face_token[0]}

                print('確認'+face_token[0])

                response = requests.post(http_url, data=data)

                req_con = response.content.decode('utf-8')
                req_dict = JSONDecoder().decode(req_con)

                print(response)
                print(response.text)

                #寫入資料程序(成功上傳再寫入)
                with lite.connect('av_data.sqlite') as db:
                    write.to_sql('face_search',con=db,if_exists='append')

                with lite.connect('av_data.sqlite') as db:
                    read=pandas.read_sql_query('select * from face_search',con = db)
                    print('網路人臉數量:'+str(eval(response.text)["face_count"]))
                    print('伺服器人臉數量:'+str(len(read['檔名'].tolist())))

                decide=0
            except KeyError as e3:
                count=count+1
                print('field')
                if count==8:
                    decide=0

        #電腦顯示存檔位置
        print('以成功'+file_end)

    else:
        #電腦告知錯誤人數
        print('圖片人數超過一人')
        os.remove(file_end)
        print('已刪除')

    #建立女優資料##################################
    
    #避免伺服器當掉 休息
    time.sleep(3)
    
    
    

In [ ]:
print('計數:'+str(excel_count))
print('伺服器長度:'+str(len(excel_data)))
print('比對是否成功:'+str(excel_count==len(excel_data)))

## 女優logo產生器

In [ ]:
import math

from imgurpython import ImgurClient#上傳圖片程序
import time

#畫圖程序
import matplotlib.pyplot as plt
import numpy as np
#畫圖程序

import example#c++連結
import ctypes
import pandas
import requests
from bs4 import BeautifulSoup

import googlemaps
from datetime import datetime

from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError,LineBotApiError
)
from linebot.models import (
    ImageCarouselColumn,ImageCarouselTemplate,CarouselColumn,CarouselTemplate,ImageMessage,MessageEvent, TextMessage, TextSendMessage,LocationMessage,FollowEvent,PostbackEvent,ImageSendMessage,FollowEvent,TemplateSendMessage,ButtonsTemplate,PostbackTemplateAction,MessageTemplateAction,URITemplateAction
)

#圖片辨識流程by face++
import requests
import pandas
import sqlite3 as lite
from json import JSONDecoder
#圖片辨識流程by face++

import os,shutil,cv2#資料移動相關

In [ ]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)

In [ ]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)
            
    av_name=list(set(read[read['logo']=='']['名子'].tolist()))
    

In [ ]:
av_name

In [ ]:
#新增全部
web=''
for item in av_name:
    if web!='no':
        print('目前女優:'+item+'\n')
        print("{0}".format('https://www.google.com.tw/search?q='+item+'+dmm'+'&source=lnms&tbm=isch')+'\n')
        web=input('logo網址:')
        if web!='no':
            with lite.connect('av_data.sqlite') as db:
                db.cursor().execute('UPDATE face_search set logo = "'+web+'" where 名子="'+item+'"')


In [ ]:
#只單純新增一人
item=input('輸入女優:')
print('目前女優:'+item+'\n')
print("{0}".format('https://www.google.com.tw/search?q='+item+'&source=lnms&tbm=isch')+'\n')
web=input('logo網址:')
with lite.connect('av_data.sqlite') as db:
    db.cursor().execute('UPDATE face_search set logo = "'+web+'" where 名子="'+item+'"')

In [ ]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)

In [ ]:
read

## 資料庫女優檢查器

In [ ]:
av_girl=os.listdir('av_girl_data')

av_girl.remove('失敗紀錄等等資訊.txt')
av_girl.remove('Client')
av_girl.remove('sure')
av_girl.remove('not_sure')
av_girl.remove('備份')
av_girl.remove('修正錯誤之後的圖片資料(未加入)')

av_name={}
for item in av_girl:
    av_name[item]=len(os.listdir('av_girl_data/'+item))
    
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)
    
sql_name=read.groupby(by=['名子'])  
sql_name=sql_name.size().to_dict()
count=1
for item in av_name:
    try:
        if av_name[item]==sql_name[item]:
            None
        else:
            print(item+'　數量錯誤')
            print('資料夾擁有：'+str(av_name[item]))
            print('資料庫擁有：'+str(sql_name[item]))
    except KeyError as e1:
        print(item+'：沒有這個資料庫')
        
print('資料夾數量：'+str(len(av_name))+'\n資料庫數量：'+str(len(sql_name)))

## 女優辨識數量抓取器

In [ ]:
av_girl=os.listdir('av_girl_data/not_sure')

In [ ]:
av_girl[0].split('_')[0]

In [ ]:
av_girl

In [ ]:
count=int(0)
name=[]
for item in av_girl:
    
    name+=[item.split('_')[0]]
    name
        
    count+=1

In [ ]:
name

In [ ]:
myset = set(name)
for item in myset:
    print(' %s 有%d次' %(item,name.count(item)))


## 資料庫重複資料刪除器

In [ ]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)

In [ ]:
for item in read['face_tokens'].tolist():
    try:
        if sum((read['face_tokens']==item).tolist())>1:

            #刪除重複資料(全部)
            with lite.connect('av_data.sqlite') as db:
                db.cursor().execute('delete from face_search where face_tokens="'+item+'"')
            
            face_token=read[read['face_tokens']==item].iloc[0,1]
            name=read[read['face_tokens']==item].iloc[0,2]
            av_file_name=read[read['face_tokens']==item].iloc[0,3]
            pinyin=read[read['face_tokens']==item].iloc[0,4]
            ps=read[read['face_tokens']==item].iloc[0,5]
            logo=read[read['face_tokens']==item].iloc[0,6]

            av_data={'名子':[name],'羅馬拼音':[pinyin],'face_tokens':[face_token],'附註':[ps],'檔名':[av_file_name],'logo':[logo]}
            write=pandas.DataFrame(av_data)
            #將資料回存(一筆)
            with lite.connect('av_data.sqlite') as db:
                write.to_sql('face_search',con=db,if_exists='append')
    except KeyError as e:
        None

In [ ]:
with lite.connect('av_data.sqlite') as db:
    read=pandas.read_sql_query('select * from face_search',con = db)

In [ ]:
read